In [2]:
def select_distinct_data(athena_client,result_output_location):
    query = "SELECT DISTINCT * from fire_data.rfs_fire_data where state in ('NSW','VIC','QLD')"
    response = athena_client.start_query_execution(
    QueryString=query,
    ResultConfiguration={"OutputLocation": result_output_location})
    return response["QueryExecutionId"]

def distinct_data_df(s3_client,bucket,response_path,pd,Point,gpd):
    # retrieve data from query
    obj = s3_client.get_object(Bucket= bucket, Key= response_path) 
    # get object and file (key) from bucket
    df = pd.read_csv(obj['Body']) # 'Body' is a key word
    df= df.dropna()
    #add lat and long as coords list to df
    df['coords'] = list(zip(df['longitude'],df['latitude']))
    #using shapely package convert coords into a POINT 
    df['coords'] = df['coords'].apply(Point)
    #convert df into GeoDF
    df = gpd.GeoDataFrame(df, geometry='coords',crs='EPSG:4326')
    return df

def points_in_polys_join(gpd,fires_df):
    population_geometry_df= gpd.read_file('lga_population_geometry/lga_population_geometry.geojson')
    pointInPolys = gpd.tools.sjoin(fires_df, population_geometry_df, predicate="within", how='inner')
    return pointInPolys

def popup_html(row,pointInPolys):
    report_date=pointInPolys['report_date'].iloc[row] 
    area=pointInPolys['lga_name'].iloc[row]
    left_col_color = "#3e95b5"
    right_col_color = "#f2f9ff"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="100px">{}</h4>""".format(area) + """</center>
<center> <table style="height: 26px; width: 250px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;padding: 10px">Report Date </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";padding-left: 30px;">"""+report_date + """</td>
</table></center>
</html>
"""
    return html

def map_data():
    fire_location = pointInPolys_df[["latitude", "longitude"]]
    map = folium.Map(location=[fire_location.latitude.mean(), fire_location.longitude.mean()],min_zoom = 3.5)
    for row in range(0,len(pointInPolys_df)):
        html = popup_html(row,pointInPolys_df)
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.Marker([pointInPolys_df['latitude'].iloc[row],pointInPolys_df['longitude'].iloc[row]],
                      popup=popup,icon=folium.Icon(color="orange",icon = "fire")).add_to(map)

    sw = pointInPolys_df[['latitude', 'longitude']].min().values.tolist()
    ne = pointInPolys_df[['latitude', 'longitude']].max().values.tolist()
    map.fit_bounds([sw, ne])


    bins = [0, 2000, 5000, 10000, 100000, 2000000]
    columns = ['lga_code','population']
    choropleth = folium.Choropleth(
            geo_data='lga_population_geometry/lga_population_geometry.geojson',
            name='choropleth',
            data=pd.read_csv('lga_population/lga_population.csv',header= None, names= columns),
            columns=['lga_code','population'], 
            key_on='feature.properties.lga_code', 
            fill_color='PuBuGn',
            nan_fill_color="White", 
            fill_opacity=0.7,
            line_opacity=0.2,
            highlight=True,
            line_color='black',
            bins = bins,
    ).add_to(map)

    #remove the legend due to aesthetics.# branca element package
    for key in choropleth._children:
        if key.startswith('color_map'):
            del(choropleth._children[key])

    choropleth.add_to(map)

    choropleth.geojson.add_child(folium.features.GeoJsonTooltip(
            fields=['lga_name','population'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))) 
    display(map)

